# Cantilever plate 
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/shell/cantilever.ipynb)

A plate of dimensions 100mm x 50mm x 0.5mm clamped at one end. It is subjected to concentrated forces of 5N at each corner at the tip. 

A reference solution is computed with ABAQUS for the following shell elements: 

| Type   | u_max | S_max |
|--------|-------|-------|
| S3     | 30.36 | 423.6 |
| STRI3  | 30.64 | 461.7 |
| STRI65 | 30.90 | 481.4 |

In [1]:
import meshzoo
import torch

from torchfem import Shell
from torchfem.materials import IsotropicElasticityPlaneStress

## Model setup

In [2]:
# Material parameters
material = IsotropicElasticityPlaneStress(E=200000.0, nu=0.3)

# Define nodes and element of the plate
X = torch.linspace(0.0, 100.0, 10)
Y = torch.linspace(0.0, 50.0, 5)
points, cells = meshzoo.rectangle_tri(X, Y, variant="zigzag")
N = len(points)
nodes = torch.hstack([torch.tensor(points, dtype=torch.float32), torch.zeros((N, 1))])
elements = torch.tensor(cells.tolist())

# Define Shell model
cantilever = Shell(nodes, elements, material)

# Define masks for boundaries
left = nodes[:, 0] < 0.01
corner_1 = (nodes[:, 0] > 99.9) & (nodes[:, 1] < 0.01)
corner_2 = (nodes[:, 0] > 99.9) & (nodes[:, 1] > 49.99)

# Apply load boundary conditions
cantilever.forces[corner_1, 2] = 5
cantilever.forces[corner_2, 2] = 5

# Apply displacement boundary conditions
cantilever.constraints[left] = True

# Thickness
cantilever.thickness[:] = 0.5

## Solve and extract maxima

In [3]:
# Solve
u, f = cantilever.solve()

# Compute stress and maxima
mises = cantilever.compute_stress(u, z=0.25, mises=True)
print(f"u_max: {torch.max(u[:, 0:3]):.2f}, S_max: {torch.max(mises):.1f}.")

u_max: 30.70, S_max: 430.4.


## Visualize displacement

In [4]:
# Displacement on deformed configuration
cantilever.plot(u[:, 0:3], node_property={"u": torch.linalg.norm(u[:, 0:3], dim=1)})

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Visualize v. Mises stress

In [5]:
# Mises stress on deformed configuration
cantilever.plot(u[:, 0:3], element_property={"mises": mises}, thickness=True)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…